In [1]:
import json
import numpy as np
from model_balancing_cvx import ModelBalancingConvex, Q_
import cvxpy as cp
np.set_printoptions(precision=1)

In [2]:
with open("../cvxpy/examples/double-branch-model/no_noise_with_noisefree_kinetic_data/double_branch_model_no_noise_with_noisefree_kinetic_data.json", "rt") as fp:
    data = json.load(fp)

args = {}
args["S"] = np.array(data["network"]["stoichiometric_matrix"])
args["A_act"] = np.array(data["network"]["activation_matrix"])
args["A_inh"] = np.array(data["network"]["inhibition_matrix"])
args["fluxes"] = Q_(data["reaction_fluxes"]["data"]["mean"], data["reaction_fluxes"]["unit"])

args["Keq_gmean"] = Q_(data["kinetic_constants"]["Keq"]["combined"]["geom_mean"], "")
args["Keq_gmean"][2] *= 1000  # fixing standard concentration convension (from 1 mM to 1 M)
args["Keq_ln_cov"] = np.array(data["kinetic_constants"]["Keq"]["combined"]["cov_ln"])

args["kcatf_gmean"] = Q_(data["kinetic_constants"]["Kcatf"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatf"]["unit"])
args["kcatf_ln_cov"] = np.array(data["kinetic_constants"]["Kcatf"]["combined"]["cov_ln"])

args["kcatr_gmean"] = Q_(data["kinetic_constants"]["Kcatr"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatr"]["unit"])
args["kcatr_ln_cov"] = np.array(data["kinetic_constants"]["Kcatr"]["combined"]["cov_ln"])

args["Km_gmean"] = Q_(data["kinetic_constants"]["KM"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KM"]["unit"])
args["Km_ln_cov"] = np.array(data["kinetic_constants"]["KM"]["combined"]["cov_ln"])

args["Ka_gmean"] = Q_(data["kinetic_constants"]["KA"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KA"]["unit"])
args["Ka_ln_cov"] = np.array(data["kinetic_constants"]["KA"]["combined"]["cov_ln"])

args["Ki_gmean"] = Q_(data["kinetic_constants"]["KI"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KI"]["unit"])
args["Ki_ln_cov"] = np.array(data["kinetic_constants"]["KI"]["combined"]["cov_ln"])

args["conc_met_gmean"] = Q_(data["metabolite_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_met_gstd"] = np.array(data["metabolite_concentrations"]["combined"]["geom_std"])

args["conc_enz_gmean"] = Q_(data["enzyme_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_enz_gstd"] = np.array(data["enzyme_concentrations"]["combined"]["geom_std"])

args["rate_law"] = "CM"

In [3]:
mb = ModelBalancingConvex(**args)
mb.initialize_with_gmeans()
print("kcat reverse comparison:")
print("from JSON:  ", args["kcatr_gmean"].m_as("1/s"))
print("calculated: " , np.exp(mb.ln_kcatr.value))

kcat reverse comparison:
from JSON:   [1408.7    8.8  362.3   53.   528.1]
calculated:  [1408.1    8.8  362.5   53.   528.2]


In [4]:
print("enzyme concentration comparison:")
print("from JSON:  \n", args["conc_enz_gmean"].m_as("M"))
print("calculated: \n" , np.exp(mb.ln_conc_enz.value))

enzyme concentration comparison:
from JSON:  
 [[6.4e-07 9.3e-07 9.4e-07 1.1e-06]
 [2.1e-06 1.2e-06 1.0e-06 8.4e-07]
 [1.0e-06 9.0e-07 6.1e-07 5.6e-07]
 [5.0e-07 4.9e-07 3.8e-07 6.0e-07]
 [1.4e-06 6.4e-07 5.1e-07 3.7e-07]]
calculated: 
 [[6.8e-07 1.0e-06 9.9e-07 1.1e-06]
 [2.2e-06 1.3e-06 1.1e-06 8.9e-07]
 [1.1e-06 9.3e-07 6.3e-07 5.8e-07]
 [5.2e-07 5.1e-07 4.0e-07 6.3e-07]
 [1.5e-06 6.7e-07 5.4e-07 3.9e-07]]


In [5]:
%%time

mb = ModelBalancingConvex(**args)
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

optimized total squared Z-scores = 0.054
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.01
kcatf = 0.03
kcatr = 0.0
Km = 0.01
Ka = 0.0
Ki = 0.0
CPU times: user 8.53 s, sys: 3.43 s, total: 12 s
Wall time: 11 s


In [6]:
%%time

mb = ModelBalancingConvex(**args)

print("Setting enzyme conc. and kcatr gmeans to calculated values...")
# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

Setting enzyme conc. and kcatr gmeans to calculated values...
initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0
CPU times: user 9.13 s, sys: 2.41 s, total: 11.5 s
Wall time: 10.3 s
